# mbnpy Tutorial 00: What and why is matrix-based Bayesian network (MBN)?

We use the reliability building block (RBD) example illustrated in Byun et al. (2019).

## Motivation 
### What is Bayesian network (BN)?

Bayesian network (BN) is a probabilistic graphical model, whose graphical representation of dependence between many random variables greatly facilitates high-dimensional probabilistic modelling. In a BN graph, random variables are represented by circular nodes, and their dependence is represented by directed arrows. For more information, I strongly recommend Koller (2009).

For example, let us consider the example BN in Fig. 1. The BN represents a system event subjected to some hazard risk. In the graph, random variables $H$, $X_1, \cdots, X_N$, and $S$ respectively represent intensity of a hazard event, component events, and a system event. 

One of the best advantages of BN is that it enables causal relationship between variables to be readily formulated using directed arrows. In Fig. 1, hazard $H$ affects performance of components $X_1,\cdots,X_N$, which then determine that of the system $S$.

Once a BN graph is set up, each node is assigned a conditional probability on their parent nodes. For instance, the BN in Fig. 1 has $P(H)$, $P(X_1|H),\cdots,P(X_N|H)$, and $P(S|X_1, \cdots, X_N)$.

Then, the joint probability represented by the BN is simply the product of all these variables, i.e.
$$P(S,X_1, \cdots, X_N,H)=P(S|X_1, \cdots, X_N)\cdot P(X_1|H)\cdot \cdots \cdot P(X_N|H)\cdot P(H).$$

<figure>
<img src="figs/why_mbn.png" alt="fig:why-mbn" style="width:400px">
<figcaption align = "left"> Fig. 1 An example BN representing a system subjected to hazard risks </figcaption>
</figure>

### Challenge: unaffordable memory

The converging structure between $X_1, \cdots, X_N$ and $S$ commonly happens when dealing with system events since a system's state by definition depends on components' states. And this is where the primary challenge arises because one needs to quantify the probability distribution $P(S|X_1,\cdots,X_N)$, whose dimension is as high as $(N+1)$. If we have 30 component events and a system event, each having a binary state, we end up having to store $2^{31}\approx 10^{9.3}$ probabilities, which is simply impossible. 

### Opportunity: Regularity in system events

However, there are some ways for better use of memory. For example, system functions are often deterministic and have regularity that can be encoded using much less memory than the actual number of existing instances. For example, consider the reliability block diagram (RBD) in Fig. 2, which was used in Byun et al. (2019). The system consists of 8 component events, represented by random variables $X_1,\cdots,X_8$. Each $X_n$, $n=1,\cdots,8$ takes a binary state of 0 (failure) and 1 (survival). Then, the system performance function is defined as whether node $s$ and node $t$ are connected (1 for connected and 0 otherwise). 

<figure>
<img src="figs/rbd.png" alt="fig:rbd" style="width:500px">
<figcaption align = "left"> Fig. 2 An example RBD </figcaption>
</figure>

This means that there are $2^{(8+1)}=512$ instances in total. However, if one applies branch and bound to decompose the event space, one can describe the whole event space by 9 branches, which is illustrated in Fig. 3 (for more information, refer to Byun et al. (2019)). For example, the leftmost branch at the top refers to that given $(X_8 = 0)$, the system fails, i.e. $s^0$ (which is an abbreviation of $S=0$), regardless of the states of other components. In other words, this single branch represents $2^7=128$ instances (over $X_1,\cdots,X_7$).

<figure>
<img src="figs/rbd_bnb.png" style="width:500px">
<figcaption align = "left"> Fig. 3 Branch and bound of the RBD in Fig. 2 </figcaption>
</figure>


### Matrix-based Bayesian network (MBN): An alternative data structure of BN

MBN has been proposed as an alternative data structure of BN to handle large-scale systems (Byun et al. 2019; Byun and Song 2021). In particular, it provides a means to encode the challenging distribution $P(S|X_1,\cdots,X_N)$ by utilising regularity in a system event. An example of such regularities is the branch and bound result in Fig. 3. In the following demonstration, I will show how MBN can be used for encoding the branch and bound result.

## Demonstration: reliability block diagram (RBD)

I will explain using the example RBD in Fig. 2, which was used in Byun et al. (2019). In the problem, the system is subjected to a hazard risk, for which one needs to calculate the system failure probability (i.e. the probability of node $s$ and $t$ being disconnected) given an occurrence of a hazard event.

For the following calculations, one first needs to import mbnpy toolkit. Below, we are assuming that conda environment <b>bns</b> has been activated already--this can be done by using the file <b>BNS_JT_py3.9.yml</b> available at the repository <b>https://github.com/jieunbyun/BNS-JT</b>.

In [5]:
from BNS_JT import cpm, variable
import numpy as np

### Quantification of probability distributions

#### (1) Hazard node

Random variable $H$ represents insensity of a hazard event. It has 3 states, 0 (mild), 1 (moderate), and 2 (intense), with probability distribution
$$P(h^0)=0.6, P(h^1)=0.3, P(h^2)=0.1,$$ 
where $h^k := (H=k)$.

In [6]:
cpms = {} # conditional probability matrix (CPM), which refers to probability distribution in MBN setting
varis = {} # information of variables

varis['haz'] = variable.Variable(name = 'haz', B=np.eye(3), values=['mild', 'moderate', 'intensity'] )
cpms['haz'] = cpm.Cpm( variables=[varis['haz']], no_child=1, C=np.array([0, 1, 2]), p = [0.6, 0.3, 0.1] )

There are a few things to note here about mbnpy.

First, in the B matrix, which is illustrated in Byun and Song (2021), each row represents each of *basic* and *composite* state, and each column represents each <em>basic</em> state. <br>
For example, consider a B matrix in Fig. 4. There are three basic states as there are three columns, and five states in total being represented by the five rows. <br>
Then, in each row, the basic state(s) represented by a state has its column value as 1 and otherwise, 0. By construction, the uppermost part is always an identity matrix, and following rows represent a composite state. <br>
In the example below, the composite state 4 represents either 1 or 2. <br>
The necessity of composite states will become clearer when handling the system event. 

<figure>
<img src="figs/what_b.png" style="width:600px">
<figcaption align = "left"> Fig. 4 Illustration of B matrix </figcaption>
</figure>

Second, <b>Variable</b> takes its third input as values, which should be the list with the same length as the number of basic states. It is simply to record what each state means for convenience.

Third, <b>Cpm</b> takes in its first input as a list of variables in its scope and second variable as the number of child nodes among them. Note that it is regarded that the first *no_child* variables in *variables* are considered child nodes and the rest are parent (or conditional) nodes. 

Fourth, the third and the fourth inputs of <b>Cpm</b> are respectively states and probabilities of each instance. The same rows of <b>C</b> and <b>p</b> must indicate the same instance.

#### (2) Component events

The component event $X_n$, $n=1,\cdots,8$, takes a binary state, 0 (failure) and 1 (survival). The probability distributions are set as
$$P(x^0|h^0)=0.05, P(x^1|h^0)=0.95, P(x^0|h^1)=0.10, P(x^1|h^1)=0.90, P(x^0|h^2)=0.20, P(x^1|h^2)=0.80, \text{\ for \ } n=1,2,3, \text{\ and \ }$$
$$P(x^0|h^0)=0.01, P(x^1|h^0)=0.99, P(x^0|h^1)=0.05, P(x^1|h^1)=0.95, P(x^0|h^2)=0.10, P(x^1|h^2)=0.90, \text{\ for \ } n=4,5,6,7,8.$$

This can be coded as below. It is noted that now we introduce a composite state 2 that represents either 0 or 1 using the B matrix.


In [7]:
no_comp = 8

for n in range(3):
    x_name = 'x'+str(n+1)
    varis[x_name] = variable.Variable(name = x_name, B=np.vstack([np.eye(2), [1, 1]]), values=['fail', 'survive'] )
    cpms[x_name] = cpm.Cpm( variables=[varis[x_name], varis['haz']], no_child=1, C=np.array([[0, 0], [1, 0], [0, 1], [1,1], [0,2], [1,2]]), p = [0.05,0.95,0.10,0.90,0.20,0.80] )

for n in range(3,8):
    x_name = 'x'+str(n+1)
    varis[x_name] = variable.Variable(name = x_name, B=np.vstack([np.eye(2), [1, 1]]), values=['fail', 'survive'] )
    cpms[x_name] = cpm.Cpm( variables=[varis[x_name], varis['haz']], no_child=1, C=np.array([[0, 0], [1, 0], [0, 1], [1,1], [0,2], [1,2]]), p = [0.01,0.99,0.05,0.95,0.10,0.90] )

print( cpms['x1'] )
print( cpms['x4'] )

Cpm(variables=['x1', 'haz'], no_child=1, C=[[0 0]
 [1 0]
 [0 1]
 [1 1]
 [0 2]
 [1 2]], p=[[0.05]
 [0.95]
 [0.1 ]
 [0.9 ]
 [0.2 ]
 [0.8 ]]
Cpm(variables=['x4', 'haz'], no_child=1, C=[[0 0]
 [1 0]
 [0 1]
 [1 1]
 [0 2]
 [1 2]], p=[[0.01]
 [0.99]
 [0.05]
 [0.95]
 [0.1 ]
 [0.9 ]]


#### (3) System event

To encode the system event's distribution, the branch and bound result in Fig. 3 can be used as below. Note that this is where the composite state 2 of $X_n$'s is necessary.

In [8]:
varis['sys'] = variable.Variable( name='sys', B=np.eye(2), values=['fail', 'survive'] )

Cs = np.array([[0, 2, 2, 2, 2, 2, 2, 2, 0], [0, 2, 2, 2, 2, 2, 2, 0, 1], [1, 1, 2, 2, 2, 2, 2, 1, 1],
               [1, 0, 1, 2, 2, 2, 2, 1, 1], [1, 0, 0, 1, 2, 2, 2, 1, 1], [0, 0, 0, 0, 0, 2, 2, 1, 1],
               [0, 0, 0, 0, 1, 0, 2, 1, 1], [0, 0, 0, 0, 1, 1, 0, 1, 1], [1, 0, 0, 0, 1, 1, 1, 1, 1]])
ps = np.array( [1,1,1,1,1,1,1,1,1] )

cpms['sys'] = cpm.Cpm(variables=[varis['sys']]+[varis['x'+str(n+1)] for n in range(no_comp)], no_child=1, C=Cs, p=ps)

print(cpms['sys'])

Cpm(variables=['sys', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'], no_child=1, C=[[0 2 2 2 2 2 2 2 0]
 [0 2 2 2 2 2 2 0 1]
 [1 1 2 2 2 2 2 1 1]
 [1 0 1 2 2 2 2 1 1]
 [1 0 0 1 2 2 2 1 1]
 [0 0 0 0 0 2 2 1 1]
 [0 0 0 0 1 0 2 1 1]
 [0 0 0 0 1 1 0 1 1]
 [1 0 0 0 1 1 1 1 1]], p=[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In the CPM (represented by <b>Cs</b> and <b>ps</b>), each row represents each branch in Fig. 3 from left to right. <br>
For example, the first row represents that the system event fails, i.e. $s^0$, given the failure of $X_8$, i.e. $x_8^0$, and all other component states do not matter, which is represented by $x_n^2$ for $n=1,\cdots,7$. <br>
Using CPM, the 512 instances can be represented by $9*(1+8) + 9*1=90$ elements (i.e. (elements in <b>Cs</b>)+(elements in <b>ps</b>)). The memory efficiency tends to be more dramatic as the number of component events increases.

In the code above, there are two noteworthy points in regard to quantifying a CPM. <br>
First, the instances represented by each row must be mutually exclusive so that they will not be double counted during the inference. <br>
Second, instances with zero probability can be excluded from a CPM because they do not affect the inference result. For example, the counterpart instance of the first row has zero probability, i.e. $P(s^1|x_8^0)=1$, so it is not included in the CPM.

## References
Koller, D. & Friedman, N. (2009). *Probabilistic graphical models: principles and techniques.* MIT press.
[1] Byun, J. E., Zwirglmaier, K., Straub, D., & Song, J. (2019). Matrix-based Bayesian Network for efficient memory storage and flexible inference. *Reliability Engineering & System Safety*, 185, 533-545. <br>
[2] Byun, J. E., & Song, J. (2021). Generalized matrix-based Bayesian network for multi-state systems. *Reliability Engineering & System Safety*, 211, 107468.
